### Importações

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

from sklearn.ensemble import IsolationForest

### Criando o DF

In [ ]:
%reload_ext autoreload
sys.path.append(os.path.abspath('..'))
from functions.clean_df import load_and_combine_csvs,clean_dataframe
from functions.state_imput import apply_state_estimation
from functions.feature_engineering import feature_engineering
from functions.preprocessing import get_preprocessor
from functions.models import run_if_normal, run_if_classified

In [ ]:
csv_path = '../raw_data'

df = load_and_combine_csvs(csv_path)
df_clean = clean_dataframe(df)
df_clean = apply_state_estimation(df_clean)


In [ ]:
df_clean = feature_engineering(df_clean)

In [ ]:
df_clean.head()

### Feature Engineering para Isolation Forest

In [ ]:
def add_frequency_features(df, cols_categoria):
    """
    Adiciona colunas de frequência normalizada para cada coluna categórica.
    Ex: 'NOME ÓRGÃO' -> 'FREQ_NOME ÓRGÃO'
    """
    df = df.copy()

    for col in cols_categoria:
        freq_map = df[col].value_counts(normalize=True)
        df[f'FREQ_{col}'] = df[col].map(freq_map)

    return df

In [ ]:
def add_monthly_ratio_features(df):
    """
    Cria duas features:
    - MEDIA_VALOR_ORGAO_MES_LOG: média do órgão no ano/mês (log)
    - RATIO_MES: valor da transação dividido pela média original (não log)
    """
    df = df.copy()

    # média original
    df['MEDIA_VALOR_ORGAO_MES'] = (
        df.groupby(['NOME ÓRGÃO', 'ANO EXTRATO', 'MÊS EXTRATO'])['VALOR TRANSAÇÃO']
          .transform('mean')
          .round(2)
    )

    # média em log (evitando log(0))
    df['MEDIA_VALOR_ORGAO_MES'] = (df['MEDIA_VALOR_ORGAO_MES'] + 1).apply(np.log)

    # ratio usando a média original
    df['RATIO_MES'] = df['VALOR TRANSAÇÃO'] / df['MEDIA_VALOR_ORGAO_MES']

    return df

In [ ]:
cols_categorica = ['NOME ÓRGÃO', 'ESTADO_ESTIMADO', 'NOME FAVORECIDO']
df_clean = add_frequency_features(df_clean, cols_categorica)

In [ ]:
df_clean = add_monthly_ratio_features(df_clean)

### Aplicando Processamento de Dados

In [ ]:
df_clean_sigiloso0 = df_clean[df_clean['SIGILOSO'] == 0].copy()
df_clean_sigiloso1 = df_clean[df_clean['SIGILOSO'] == 1].copy()

In [ ]:
preprocessor = get_preprocessor()
preprocessor.fit(df_clean)

X_scaled_sigiloso0 = preprocessor.transform(df_clean_sigiloso0)
X_scaled_sigiloso1 = preprocessor.transform(df_clean_sigiloso1)

### Treinando o Modelo

In [ ]:
iso0 = IsolationForest(contamination=0.01, random_state=42)

iso0.fit(X_scaled_sigiloso0)
df_clean_sigiloso0['IF_LABEL'] = iso0.predict(X_scaled_sigiloso0)
df_clean_sigiloso0['IF_SCORE'] = iso0.score_samples(X_scaled_sigiloso0)

In [ ]:
iso1 = IsolationForest(contamination=0.01, random_state=42)

iso1.fit(X_scaled_sigiloso1)
df_clean_sigiloso1['IF_LABEL'] = iso1.predict(X_scaled_sigiloso1)
df_clean_sigiloso1['IF_SCORE'] = iso1.score_samples(X_scaled_sigiloso1)

In [ ]:
anomalias_sigiloso0 = df_clean_sigiloso0[df_clean_sigiloso0['IF_LABEL'] == -1]
len(anomalias_sigiloso0)

In [ ]:
anomalias_sigiloso1 = df_clean_sigiloso1[df_clean_sigiloso1['IF_LABEL'] == -1]
len(anomalias_sigiloso1)

In [ ]:
anomalias_sigiloso1['DATA TRANSAÇÃO'].unique()

In [ ]:
anomalias_sigiloso1.head()

### Visualização

In [ ]:
plt.figure(figsize=(14, 7))
sns.scatterplot(
    data=df_clean_sigiloso0,
    x='DATA TRANSAÇÃO',
    y='VALOR TRANSAÇÃO',
    hue='IF_LABEL',
    palette={1: 'lightgrey', -1: 'red'},
    alpha=0.6,
    size='IF_LABEL',
    sizes=(20, 50)
)
plt.yscale('log')
plt.title('Baseline Isolation Forest: Detecção de Anomalias (Vermelho)')
plt.ylabel('Valor da Transação (Log)')
plt.xlabel('Data')
plt.legend(title='Status (1=Ok, -1=Suspeito)')
plt.show()

In [ ]:
df_plot = df_clean_sigiloso1.reset_index()

plt.figure(figsize=(14, 7))
sns.scatterplot(
    data=df_plot,
    x=df_plot.index,
    y='VALOR TRANSAÇÃO',
    hue='IF_LABEL',
    palette={1: 'lightgrey', -1: 'red'},
    alpha=0.6,
    size='IF_LABEL',
    sizes=(20, 50)
)
plt.yscale('log')
plt.title('Baseline Isolation Forest: Detecção de Anomalias (Vermelho)')
plt.ylabel('Valor da Transação (Log)')
plt.xlabel('Índice da Linha')
plt.show()

In [ ]:
len(df_clean_sigiloso0), len(df_clean_sigiloso1)

### Top suspeitos

In [ ]:
cols_view = ['NOME ÓRGÃO', 'NOME FAVORECIDO', 'VALOR TRANSAÇÃO', 'DATA TRANSAÇÃO', 'ESTADO_ESTIMADO', 'IF_SCORE']
top_anomalias_sigiloso0 = df_clean_sigiloso0[df_clean_sigiloso0['IF_LABEL'] == -1].sort_values('IF_SCORE', ascending=True).head(100)
top_anomalias_sigiloso0.head()[cols_view]

In [ ]:
cols_view = ['NOME ÓRGÃO', 'NOME FAVORECIDO', 'VALOR TRANSAÇÃO', 'DATA TRANSAÇÃO', 'ESTADO_ESTIMADO', 'IF_SCORE']
top_anomalias_sigiloso1 = df_clean_sigiloso1[df_clean_sigiloso1['IF_LABEL'] == -1].sort_values('IF_SCORE', ascending=True).head(100)
top_anomalias_sigiloso1.head()[cols_view]

### Teste da função

In [ ]:
resultado_s0 = run_if_normal(df_clean)

resultado_s0.head(2)

In [ ]:
resultado_s1 = run_if_classified(df_clean)

resultado_s1.head(2)